In [487]:
%matplotlib inline
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
import os
import cv2
import pandas as pd
import gc
import numpy as np
from tqdm import tqdm_notebook, tqdm

In [558]:

def f2cat(filename: str) -> str:
    return filename.split('.')[0]

def list_all_categories():
    files = os.listdir(os.path.join(INPUT_DIR, 'train_simplified'))
    return sorted([f2cat(f) for f in files], key=str.lower)
def apk(actual, predicted, k=3):
    """
    Source: https://github.com/benhamner/Metrics/blob/master/Python/ml_metrics/average_precision.py
    """
    if len(predicted) > k:
        predicted = predicted[:k]
    score = 0.0
    num_hits = 0.0
    for i, p in enumerate(predicted):
        if p in actual and p not in predicted[:i]:
            num_hits += 1.0
            score += num_hits / (i + 1.0)
    if not actual:
        return 0.0
    return score / min(len(actual), k)

def mapk(actual, predicted, k=3):
    """
    Source: https://github.com/benhamner/Metrics/blob/master/Python/ml_metrics/average_precision.py
    """
    return np.mean([apk(a, p, k) for a, p in zip(actual, predicted)])

def preds2catids(predictions):
    return pd.DataFrame(np.argsort(-predictions, axis=1)[:, :3], columns=['a', 'b', 'c'])

def top_3_accuracy(y_true, y_pred):
    return top_k_categorical_accuracy(y_true, y_pred, k=3)

In [569]:
#DP_DIR = './input/shuffle-csvs/'
INPUT_DIR = './input/'

prob_dir = './probs/'
ensemble_dir = './ensemble_prob/'
sub_dir = './subs/'
def load_npy(dirname):
    cls_npy_list = []
    filenames = os.listdir(dirname)
    for filename in filenames:
        full_filename = os.path.join(dirname, filename)
        ext = os.path.splitext(full_filename)[-1]
        if ext == '.npy' :#and filename.find('densenet121')>0:# and filename.find(select_prefix2)<0: 
            print(filename)
            cls_npy_list.append(full_filename)
    return cls_npy_list

cls_npys = load_npy(prob_dir)
print(len(cls_npys))

clsnp = []
local_score = []
names = []
for idx, cls in enumerate(tqdm_notebook(cls_npys)):
    score = cls.split('_')[-1].replace('.npy','')
    names.append(((cls.split('_')[0]).split('/')[-1])[:6]+score[:4])
    local_score.append(int(score))
    #print(score)
    temp = np.load(cls)
    clsnp.append(temp)
    


InceptionResNetV2_is_(256, 256, 3)_bs_40_lf_None_vf_99_lr_0.0002_cum_30_timgs_24M_localT3acc_9501.npy
rawInceptionResNetV2_is_(139, 139, 3)_bs_180_lf_None_vf_66_lr_0.002_cum_500_timgs_4M_localT3acc_9516.npy
rawInceptionResNetV2_is_(139, 139, 3)_bs_180_lf_None_vf_66_ur_True_timgs_32M_localT3acc_9516.npy
rawInceptionResNetV2_is_(256, 256, 3)_bs_40_lf_None_vf_9_lr_0.002_ur_True_cum_30_timgs_46M_localT3acc_9473.npy
rawXception_is_(128, 128, 3)_bs_128_lf_None_vf_33_lr_0.002_ur_True_cum_1000_timgs_12M_localT3acc_9474.npy
rawXception_is_(256, 256, 3)_bs_32_lf_None_vf_99_lr_0.002_cum_30_timgs_36M_localT3acc_9441.npy
ResNet50_is_(256, 256, 3)_bs_50_lf_None_vf_0_lr_0.0002_cum_30_timgs_40M_localT3acc_9457.npy
7


In [570]:
def get_score_w(local_score):
    ls = np.array(local_score)
    sub = ls - ls.min()
    div = sub/sub.max()
    add = div + 0.1
    nor = add/add.max()
    sqr = nor*nor
    print(sqr)
    return sqr

def get_corr_w(clsnp):
    corxlist=[]
    for idx1, cls1 in enumerate((clsnp)):
        corylist=[]
        for idx2, cls2 in enumerate((clsnp)):
            cor_max = np.corrcoef(np.argmax(clsnp[idx1],axis=1),np.argmax(clsnp[idx2],axis=1))[0][1]
            corylist.append(cor_max)
        corxlist.append(corylist)

    df = pd.DataFrame(corxlist,columns=names, index=names)
    #print(df)
    corr_w = []
    for i in range(df.shape[0]):
        count = 0
        thr = 0.95
        for v in df.values[i]:
            if v > thr :
                count+=1
        corr_w.append(1.0/count)   
    return np.array(corr_w), df

def getensemble_w(clsnp, local_score):
    score_w = get_score_w(local_score)
    for i in range(score_w.shape[0]):
        if score_w[i]==1.0:
            score_w[i] = 1.2 #max score add 20%
    corr_w = get_corr_w(clsnp)[0]
    ensemble_w = score_w*corr_w
    return ensemble_w

In [573]:
ensemble_w = getensemble_w(clsnp, local_score)
cor_df = get_corr_w(clsnp)[1]
cor_df
print('ensemble weight', ensemble_w)

sub [60 75 75 32 33  0 16]
div [0.8        1.         1.         0.42666667 0.44       0.
 0.21333333]
add [0.9        1.1        1.1        0.52666667 0.54       0.1
 0.31333333]
[0.66942149 1.         1.         0.22923783 0.24099174 0.00826446
 0.08113866]


,Incept9501,rawInc9516,rawInc9516,rawInc9473,rawXce9474,rawXce9441,ResNet9457
Incept9501,1.000000,0.938674,0.937285,0.929651,0.935064,0.924573,0.947429
rawInc9516,0.938674,1.000000,0.973477,0.938433,0.944116,0.934933,0.933779
rawInc9516,0.937285,0.973477,1.000000,0.936875,0.943053,0.932677,0.932387
rawInc9473,0.929651,0.938433,0.936875,1.000000,0.933185,0.926541,0.927044
rawXce9474,0.935064,0.944116,0.943053,0.933185,1.000000,0.939055,0.933180
rawXce9441,0.924573,0.934933,0.932677,0.926541,0.939055,1.000000,0.924680
ResNet9457,0.947429,0.933779,0.932387,0.927044,0.933180,0.924680,1.000000


ensemble weight [0.66942149 0.6        0.6        0.22923783 0.24099174 0.00826446
 0.08113866]


In [565]:
print(ensemble_w)
print(ensemble_w.sum())

[0.60551452 0.7        0.7        0.15861997 0.16885063 0.00249183
 0.04330468]
2.378781624882278


## Create Submission

In [566]:
test = pd.read_csv(os.path.join(INPUT_DIR, 'test_simplified.csv'))

In [567]:
wsum = np.sum(ensemble_w)
print(wsum)

for idx, cls in enumerate(tqdm_notebook(cls_npys)):
    temp = np.load(cls)
    if idx == 0 :
        cls_sum=temp*ensemble_w[0]
    else:
        cls_sum+=temp*ensemble_w[idx]
        
cls_sum=cls_sum/wsum
pre_fix = str(len(cls_npys))+ '_ensemble'
np.save(ensemble_dir+ pre_fix+'.npy',cls_sum)

2.378781624882278


In [568]:
#test_predictions = model.predict(x_test, batch_size=128, verbose=1)
top3 = preds2catids(cls_sum)
cats = list_all_categories()
id2cat = {k: cat.replace(' ', '_') for k, cat in enumerate(cats)}
top3cats = top3.replace(id2cat)

test['word'] = top3cats['a'] + ' ' + top3cats['b'] + ' ' + top3cats['c']
submission = test[['key_id', 'word']]
submission.to_csv(sub_dir + pre_fix + '.csv', index=False)
submission.head()
submission.shape

,key_id,word
0,9000003627287624,radio stereo remote_control
1,9000010688666847,hockey_puck bottlecap pool
2,9000023642890129,The_Great_Wall_of_China castle camel
3,9000038588854897,mountain finger The_Eiffel_Tower
4,9000052667981386,campfire fireplace fire_hydrant


(112199, 2)